In [ ]:
!pip install tensorflow --upgrade --user
!pip install keras==2.3.0 --user
!pip install imutils --user
!pip install opencv-python --user
!pip install torchvision --user

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import load_img
import seaborn as sns


# read cached daa used to have emotions as the last word, the other function names may change
from preliminary_caching import has_cached_emotions, cache_emotions, read_cached_data
from face_extractor import get_face_bounding_boxes, extract_faces

In [ ]:
train_data = read_cached_data("train")
validation_data = read_cached_data("validation")

In [ ]:
def heatmap(data,labels_x, labels_y,title, title_x,title_y):
    f = plt.figure()
    f.set_figwidth(2*len(labels_x))
    f.set_figheight(len(labels_y))
    ax = sns.heatmap(data,annot = True, xticklabels=labels_x, yticklabels=labels_y)
    plt.title(title, fontsize = 20)
    plt.xlabel(title_x, fontsize = 15) 
    plt.ylabel(title_y, fontsize = 15) 
    plt.xticks(rotation = 45)
    plt.yticks(rotation = 45)
    plt.show()
    
def bar(data,labels,title, title_x,title_y):
    plt.bar(data, labels)
    plt.title(title, fontsize = 20)
    plt.xlabel(title_x, fontsize = 15) 
    plt.ylabel(title_y, fontsize = 15) 
    plt.xticks(rotation = 45)
    plt.show()

def stacked_barchart(data, labels_x, labels_y,title, title_x,title_y):
    figure = plt.figure()
    figure.set_figwidth(10)
    figure.set_figheight(10)
    xloc = np.arange(len(labels_x))
    plt.xticks(xloc,labels_x)
    legend = []
    bottom = np.zeros(len(labels_x))
    for i in range(data.shape[0]):
        p = plt.bar(xloc,data[i],bottom = bottom, width = 0.5)
        bottom += data[i]
        legend.append(p[0])
    plt.legend(legend,labels_y)
    plt.title(title, fontsize = 20)
    plt.xlabel(title_x, fontsize = 15) 
    plt.ylabel(title_y, fontsize = 15) 
    plt.show()

In [ ]:
# emotions vs race is being shown in the next 2 figures, race is on the bottom axis
emotions = ["neutral","happiness","sadness","surprise","sad","disgust","angry","contempt"]
race4 = ["White","Black","Asian","Indian"]
train_race_emotions = np.zeros((8,4))
train_races = np.zeros(4)
train_emotions = np.zeros(8)

for key,val in train_data.items():
    emotion = val["emotion"]
    if emotion <8:
        race = val["race"]
        train_race_emotions[emotion,race] += 1
        train_races[race] += 1
        train_emotions[emotion] += 1

heatmap(train_race_emotions, race4,emotions,"Emotions by race", "Race", "Emotion")

In [ ]:
stacked_barchart(train_race_emotions,race4,emotions,"Emotions by Race", "Race", "Count")

In [ ]:
stacked_barchart(train_race_emotions.T,emotions,race4,"Race by Emotion", "Emotion", "Count")

In [ ]:
bar(emotions,train_emotions,"Emotions in Training Set", "Emotion", "Count")

In [ ]:
bar(race4,train_races,"Races in Training Set", "Race", "Count")

In [ ]:
a = 10
g = 2
r = 7
e = 8
val_race_emotions = np.zeros((e,r))
val_gender_emotions = np.zeros((e,g))
val_age_emotions = np.zeros((e,a))
val_age_genders = np.zeros((g,a))
val_age_races = np.zeros((r,a))
val_race_genders = np.zeros((g,r))
val_races = np.zeros(r)
val_emotions = np.zeros(e)
val_genders = np.zeros(g)
val_ages = np.zeros(a)
race7 = ['White','Black','Latino_Hispanic','East Asian','Southeast Asian','Indian','Middle Eastern']
genders = ["Male","Female"]
ages = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80-89","90-99"]
for val in validation_data.values():
    emotion = val["emotion"]
    if emotion <8:
        race = val["race"]
        age = val["age"]
        gender = val["gender"]
        val_race_emotions[emotion,race] += 1
        val_races[race] += 1
        val_emotions[emotion] += 1
        val_genders[gender]+=1
        val_ages[age]+=1
        val_gender_emotions[emotion,gender] +=1
        val_age_emotions[emotion,age] +=1
        val_age_genders[gender,age]+=1
        val_age_races[race,age]+=1
        val_race_genders[gender,race]+=1
heatmap(val_race_emotions, race7,emotions,"Emotions by race", "Race", "Emotion")

In [ ]:
stacked_barchart(val_race_emotions,race7,emotions,"Emotions by race", "Race", "Count")

In [ ]:
stacked_barchart(val_race_emotions.T,emotions,race7,"Races by Emotion", "Emotion", "Count")

In [ ]:
heatmap(val_gender_emotions, genders,emotions,"Emotions by Gender", "Gender", "Emotion")

In [ ]:
heatmap(val_age_emotions, ages,emotions,"Emotions by Age", "Age", "Emotion")

In [ ]:
bar(emotions,val_emotions,"Emotions in Validation Set", "Emotion", "Count")

In [ ]:
bar(genders,val_genders,"Genders in Validation Set", "Gender", "Count")

In [ ]:
bar(ages,val_ages,"Ages in Validation Set", "Age Range", "Count")

In [ ]:
stacked_barchart(val_race_genders,race7,genders,"Gender by Race", "Race", "Count")

In [ ]:
stacked_barchart(val_race_genders.T,genders,race7,"Race by Gender", "Gender", "Count")

In [ ]:
stacked_barchart(val_age_races,ages,race7,"Race by Age", "Age", "Count")

In [ ]:
stacked_barchart(val_age_races.T,race7,ages,"Age by Race", "Race", "Count")

In [ ]:
stacked_barchart(val_age_genders,ages,genders,"Gender by Age", "Age", "Count")

In [ ]:
stacked_barchart(val_age_genders.T,genders,ages,"Age by Gender", "Gender", "Count")

In [ ]:
def display_image(image_id):
    
    image = mpimg.imread(f"/data/datasets/affectNet/train_set/images/{image_id}.jpg")
    plt.imshow(image)
    plt.show()
    
def display_face_extraction(image_id):
    
    image = mpimg.imread(f"/data/datasets/affectNet/val_set/images/{image_id}.jpg")
    bounds = get_face_bounding_boxes(image, 0.0, 1)
    
    print(bounds)
    extracted_face = extract_faces(image, bounds)[0]
    
    plt.imshow(extracted_face)
    plt.show()
display_image(223999)

In [ ]:
import os
path = "/data/datasets/affectNet/train_set/images"
dir_list = os.listdir(path)
def get_excluded_image():
    for img in dir_list:
        if path+"/"+img not in train_data:
            image = mpimg.imread(path+"/"+img)
            yield image
excluded_image_generator = get_excluded_image()

In [ ]:
image = next(excluded_image_generator)
plt.imshow(image)
plt.show()

In [ ]:
print("validation samples "+str(len(validation_data)))
print("training samples "+str(len(train_data)))
avg = 0
sum = 0
for i in range(len(ages)):
    avg += (10*i+5)*val_ages[i]
    sum += val_ages[i]
avg/=sum
print("average age " + str(avg))